# İş Birlikçi Filtreleme ( Collaborative Filtering )

İş birlikçi filtrelemeyi üç temel başlıkta ele alabiliriz; 

1. Item-Based Collaborative Filtering
2. User-Based Collaborative Filtering
3. Model-Based Collaborative Filtering

### 1. Item-based collaborative filtering (ürün temelli iş birlikçi filtreleme):

- Item benzerlikleri üzerinden öneriler yapar. Buradaki benzerlik içerik benzerliği değildir.
- Örneğin, izlenen bir filmin beğenilme yapısı (puan verilme alışkanlıkları) üzerinden benzer bir beğenilme yapısına sahip başka bir filmi kullanıcıya önerme tekniğidir. 
- Kullanıcıların beğenme alışkanlıkları üzerinden filmler arasında yüksek korelasyon değerlerinden gözlemler yapılır.
- Bu gözlemlere göre en yüksek korelasyonlu filmler önerilir.
- Örneğin, mehmet bey bir filmi izlemiş ve beğenmiş olsun. Mehmet beyin beğenmiş olduğu filme benzer beğenilme örüntüsü olan filmleri önerir. 

In [1]:
# Veri seti: https://grouplens.org/datasets/movielens/

# Adım 1: Veri Setinin Hazırlanması
# Adım 2: User Movie Df'inin Oluşturulması
# Adım 3: Item-Based Film Önerilerinin Yapılması
# Adım 4: Çalışma Scriptinin Hazırlanması

##  Adım 1: Veri Setinin Hazırlanması

In [2]:
import pandas as pd

In [3]:
# Verilerimizi import edelim. 
movie = pd.read_csv("/Users/elifbagci/Desktop/Miuul:DATA SCİENCE/Elif Miuul notlar/Recommendation Systems/recommender_systems/datasets/movie_lens_dataset/movie.csv")
rating= pd.read_csv("/Users/elifbagci/Desktop/Miuul:DATA SCİENCE/Elif Miuul notlar/Recommendation Systems/recommender_systems/datasets/movie_lens_dataset/rating.csv")



In [4]:
df= movie.merge(rating, how="left", on="movieId") #left join ile rating ve movieId verilerini birleştiriyoruz. 
df.head

<bound method NDFrame.head of           movieId                          title  \
0               1               Toy Story (1995)   
1               1               Toy Story (1995)   
2               1               Toy Story (1995)   
3               1               Toy Story (1995)   
4               1               Toy Story (1995)   
...           ...                            ...   
20000792   131254   Kein Bund für's Leben (2007)   
20000793   131256  Feuer, Eis & Dosenbier (2002)   
20000794   131258             The Pirates (2014)   
20000795   131260            Rentun Ruusu (2001)   
20000796   131262               Innocence (2014)   

                                               genres    userId  rating  \
0         Adventure|Animation|Children|Comedy|Fantasy       3.0     4.0   
1         Adventure|Animation|Children|Comedy|Fantasy       6.0     5.0   
2         Adventure|Animation|Children|Comedy|Fantasy       8.0     4.0   
3         Adventure|Animation|Children|Comedy

## Adım 2: User Movie Df'inin Oluşturulması

Bu noktada yaşanan problemlerden bir tanesi de seyreklik durumudur.
- Örneğin bir kullanıcı sadece bir filme puan vermiş olsun; 
Bir filme puan verdiği halde bu kullanıcı User Movie Df içerisindeki diğer filmlerde de birer hücre teşkil edecektir. Bu durum da yapılacak hesaplama işlemlerini geciktirecek ve performans problemleri ortaya çıkarmaktadır.
- İki filmi ele alalım, bu iki filmden bir tanesi binlerce puan almış bir tanesi ise 40-50 puan almış olsun. Bu iki filmde 40-50 puan almış bir filme öneride bulunmak çok doğru bir yaklaşım olmaycaktır. Bu durumu çözmek için; binden az puan almış filmleri çalışmanın dışında bırakabiliriz.

In [5]:
df["title"].nunique()
# kaç adet filmimiz olduğuna bakalım, 27262 adet eşsiz film var. 

27262

In [6]:
df["title"].value_counts().head()
# hangi filme kaç rate verildiğine bakalım. 

Pulp Fiction (1994)                 67310
Forrest Gump (1994)                 66172
Shawshank Redemption, The (1994)    63366
Silence of the Lambs, The (1991)    63299
Jurassic Park (1993)                59715
Name: title, dtype: int64

In [7]:
# binden az puan alanları analiz dışında bırakmayı "yorumladık". 

In [8]:
# yukarıdaki tabloyu bir dataframe e çeviriyoruz. 
comment_counts=pd.DataFrame(df["title"].value_counts())
comment_counts
#aşağıda filmlerin bir kısmının 1 yorum/rate aldığını görüyoruz. 

,title
Pulp Fiction (1994),67310
Forrest Gump (1994),66172
"Shawshank Redemption, The (1994)",63366
"Silence of the Lambs, The (1991)",63299
Jurassic Park (1993),59715
...,...
Rapture (Arrebato) (1980),1
"Education of Mohammad Hussein, The (2013)",1
Satanas (2007),1
Psychosis (2010),1


In [9]:
comment_counts[comment_counts["title"] <= 1000]
#bu kod ile elimizde az yoruma sahip olan filmleri belirlemiş olduk. Bunların sadece index bilgisine erişip rare_movies olarak kaydedelim. 

,title
"Bear, The (Ours, L') (1988)",999
Rosewood (1997),999
Ted (2012),999
One Night at McCool's (2001),999
Marked for Death (1990),998
...,...
Rapture (Arrebato) (1980),1
"Education of Mohammad Hussein, The (2013)",1
Satanas (2007),1
Psychosis (2010),1


In [10]:
rare_movies = comment_counts[comment_counts["title"] <= 1000].index
rare_movies
# ancak bizim bunlara değil, bunların dışında kalan filmlere ihtiyacımız var. Bunun için;

Index(['Bear, The (Ours, L') (1988)', 'Rosewood (1997)', 'Ted (2012)',
       'One Night at McCool's (2001)', 'Marked for Death (1990)',
       'Three to Tango (1999)', 'Adam's Rib (1949)',
       'I Now Pronounce You Chuck and Larry (2007)',
       'Italian for Beginners (Italiensk for begyndere) (2000)',
       'Husbands and Wives (1992)',
       ...
       'Satan's Sword (Daibosatsu tôge) (1960)',
       'Blind Massage (Tui na) (2014)', 'Prêt à tout (2014)',
       'Ditchdigger's Daughters, The (1997)', 'A.K. (1985)',
       'Rapture (Arrebato) (1980)',
       'Education of Mohammad Hussein, The (2013)', 'Satanas (2007)',
       'Psychosis (2010)', 'Innocence (2014)'],
      dtype='object', length=24103)

In [11]:
common_movies = df[~df["title"].isin(rare_movies)] #rare_movies'in içindekilere bak(isin) ve bunların dışında kalanları getir(~). 
common_movies

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,2009-01-02 01:13:41
...,...,...,...,...,...,...
19985698,114240,Aladdin (1992),Adventure|Animation|Children|Comedy|Fantasy,28195.0,4.0,2014-09-22 20:52:18
19985699,114240,Aladdin (1992),Adventure|Animation|Children|Comedy|Fantasy,51334.0,3.0,2014-09-23 15:53:39
19985700,114240,Aladdin (1992),Adventure|Animation|Children|Comedy|Fantasy,120575.0,2.5,2014-10-08 14:23:39
19985701,114240,Aladdin (1992),Adventure|Animation|Children|Comedy|Fantasy,124998.0,2.5,2014-09-20 22:16:14


In [12]:
# yorum/rate sayısı 1000den küçük olanları sildik. Peki elimizde kaç adet film kaldı ona bakalım. 
common_movies["title"].nunique()
# 27262 adet eşsiz film varken, şuan elimizde 3159 adet eşsiz film var. 

3159

In [13]:
# şimdi öyle bir işlem yapmalıyız ki satırlarda userıd stunlarda ise title'lar olsun. Yani bir pivot işlemi yapmamız lazım. 
user_movie_df= common_movies.pivot_table(index=["userId"], columns=["title"], values=["rating"])
# Yukarıda pivot table oluşturduk, satırlara userID getirtdik,
# Sütunlara film isimlerini getirdik,
# Kesişimlerine ise rating'i getirdik.

In [14]:
user_movie_df
# bu tablo bizim için hala uygun değil. Çünkü yukarda bahsedilen seyreklik durumu görülmekte. Örneğin bir kullanıcı tek bir yorum yapmış gibi veriler de mevcut. 
# bu durum için şimdilik birşey yapmayacağız ancak belirli oyu veren kullanıcılar konusunda da bir indirgeme işlemi yapılabilir. 

rating                              \
title    'burbs, The (1989) (500) Days of Summer (2009)   
userId                                                    
1.0                     NaN                         NaN   
2.0                     NaN                         NaN   
3.0                     NaN                         NaN   
4.0                     NaN                         NaN   
5.0                     NaN                         NaN   
...                     ...                         ...   
138489.0                NaN                         NaN   
138490.0                NaN                         NaN   
138491.0                NaN                         NaN   
138492.0                NaN                         NaN   
138493.0                NaN                         NaN   

                                                                       \
title    *batteries not included (1987) ...And Justice for All (1979)   
userId                                                                  
1.0                                 NaN                           NaN   
2.0                                 NaN                           NaN   
3.0                                 NaN                           NaN   
4.0                                 NaN                           NaN   
5.0                                 NaN                           NaN   
...                                 ...                           ...   
138489.0                            NaN                           NaN   
138490.0                            NaN                           NaN   
138491.0                            NaN                           NaN   
138492.0                            NaN                           NaN   
138493.0                            NaN                           NaN   

                                                             \
title    10 Things I Hate About You (1999) 10,000 BC (2008)   
userId                                                        
1.0                                    NaN              NaN   
2.0                                    NaN              NaN   
3.0                                    NaN              NaN   
4.0                                    NaN              NaN   
5.0                                    NaN              NaN   
...                                    ...              ...   
138489.0                               NaN              NaN   
138490.0                               NaN              NaN   
138491.0                               NaN              NaN   
138492.0                               NaN              NaN   
138493.0                               NaN              NaN   

                                \
title    101 Dalmatians (1996)   
userId                           
1.0                        NaN   
2.0                        NaN   
3.0                        NaN   
4.0                        NaN   
5.0                        NaN   
...                        ...   
138489.0                   NaN   
138490.0                   NaN   
138491.0                   NaN   
138492.0                   NaN   
138493.0                   3.0   

                                                                 \
title    101 Dalmatians (One Hundred and One Dalmatians) (1961)   
userId                                                            
1.0                                                     NaN       
2.0                                                     NaN       
3.0                                                     NaN       
4.0                                                     NaN       
5.0                                                     NaN       
...                                                     ...       
138489.0                                                NaN       
138490.0                                                NaN       
138491.0                                                2.5       
138492.

In [15]:
user_movie_df.shape 
# görüldüğü üzere 138493 satır var bunlar userlar.
# stünlar filmlerdir. 

(138493, 3159)

## Adım 3: Item-Based Film Önerilerinin Yapılması

In [16]:
# bu bölümde film önerilerini gerçekleştiriyor olacağız. 
# Örneğin aşağıdaki gibi bir film olsun, elimdeki user_movie dataframe ı vardı. Şimdi bir değişken seçer gibi bu dataframe e uygulasak;
movie_name = "Matrix, The (1999)"

# Bu yöntem ile önerimizi zenginleştirdik ve arkasına büyük bir topluluğun fikir birlikliğini aldık. İş birlikçi filtreleme bu demektir. 

In [ ]:
movie_name_column = user_movie_df[movie_name]
user_movie_df.corrwith(movie_name_column).sort_values(ascending=False).head(10)#user_movie_df i ile correlasyona bakıyoruz. 

In [ ]:
# Başka bir filmde yapalım bunu;
movie_name = "Ocean's Twelve (2004)"
movie_name_column = user_movie_df[movie_name]
user_movie_df.corrwith(movie_name_column).sort_values(ascending=False).head(10)

# Bu çalışmanın arka planında bir filme;

    # En benzer 1-5 arasında puan verilme davranışını gösteren diğer filmleri ön plana çıkarma vardır.

In [19]:
# Burada tabi filmin bütün ismini girmemiz gerekiyor, bunu bulabilmemiz gerekir.
# Eğer ki bulamazsak bu aramayı da yapamayız.
# Dolayısıyla bizim bir key word girdiğimizde bu key word'ü içeren filmleri bulabiliyor olmamız gerekir. Bunun için;

def check_film(keyword, user_movie_df):
    return [col for col in user_movie_df.columns if keyword in col]

In [20]:
check_film("Sherlock", user_movie_df)

[]

In [ ]:
movie_name = 'Sherlock Holmes: A Game of Shadows (2011)'
movie_name_column = user_movie_df[movie_name]
user_movie_df.corrwith(movie_name_column).sort_values(ascending=False).head(10)

## Adım 4: Çalışmanın Scriptinin Hazırlanması

In [22]:
def create_user_movie_df():
    import pandas as pd
    movie = pd.read_csv("/Users/elifbagci/Desktop/Miuul:DATA SCİENCE/Elif Miuul notlar/Recommendation Systems/recommender_systems/datasets/movie_lens_dataset/movie.csv")
    rating= pd.read_csv("/Users/elifbagci/Desktop/Miuul:DATA SCİENCE/Elif Miuul notlar/Recommendation Systems/recommender_systems/datasets/movie_lens_dataset/rating.csv")
    df = movie.merge(rating, how="left", on="movieId")
    comment_counts = pd.DataFrame(df["title"].value_counts())
    rare_movies = comment_counts[comment_counts["title"] <= 1000].index
    common_movies = df[~df["title"].isin(rare_movies)]
    user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")
    return user_movie_df

user_movie_df = create_user_movie_df()

In [29]:
user_movie_df

title,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),...,Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zulu (1964),[REC] (2007),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138489.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138490.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138491.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
def item_based_recommender(movie_name, user_movie_df):
    movie_name = user_movie_df[movie_name]
    return user_movie_df.corrwith(movie_name).sort_values(ascending=False).head(10)

item_based_recommender("Matrix, The (1999)", user_movie_df)

movie_name = pd.Series(user_movie_df.columns).sample(1).values[0]

item_based_recommender(movie_name, user_movie_df)

title
Hannah and Her Sisters (1986)     1.000000
Moll Flanders (1996)              0.662760
Annie Hall (1977)                 0.597412
Manhattan (1979)                  0.568080
Crimes and Misdemeanors (1989)    0.554662
Dallas Buyers Club (2013)         0.532851
High Art (1998)                   0.518738
Stuart Saves His Family (1995)    0.496561
Age of Innocence, The (1993)      0.485352
Caine Mutiny, The (1954)          0.478813
dtype: float64

In [24]:
def item_based_recommender(movie_name, user_movie_df):
    movie_name = user_movie_df[movie_name]
    return user_movie_df.corrwith(movie_name).sort_values(ascending=False).head(10)



In [25]:
item_based_recommender("Matrix, The (1999)", user_movie_df)



title
Matrix, The (1999)                                           1.000000
Matrix Reloaded, The (2003)                                  0.516906
Matrix Revolutions, The (2003)                               0.449588
Animatrix, The (2003)                                        0.367151
Blade (1998)                                                 0.334493
Terminator 2: Judgment Day (1991)                            0.333882
Minority Report (2002)                                       0.332434
Edge of Tomorrow (2014)                                      0.326762
Mission: Impossible (1996)                                   0.320815
Lord of the Rings: The Fellowship of the Ring, The (2001)    0.318726
dtype: float64

In [26]:
movie_name = pd.Series(user_movie_df.columns).sample(1).values[0]

item_based_recommender(movie_name, user_movie_df)
 

title
Mickey Blue Eyes (1999)                       1.000000
Two Weeks Notice (2002)                       0.519844
Quest, The (1996)                             0.513751
Jet Li's Fearless (Huo Yuan Jia) (2006)       0.513365
Terminal Velocity (1994)                      0.506830
Drop Zone (1994)                              0.506063
Prince of Persia: The Sands of Time (2010)    0.505130
Free Willy 2: The Adventure Home (1995)       0.504846
Cops and Robbersons (1994)                    0.501124
Nine Months (1995)                            0.499803
dtype: float64